# Credit Risk Resampling Techniques

In [128]:
import warnings
warnings.filterwarnings('ignore')

In [129]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [130]:
#added
from sklearn.model_selection import train_test_split

# Read the CSV into DataFrame

In [131]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [132]:
# Create our features
from sklearn.preprocessing import StandardScaler

#have i chosen the wrong columns? "loan_status"
#regression and the counters output are not right

#X
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)
X = pd.get_dummies(X)
X.head()

# Create our target

#y = df["loan_status"].ravel()
y = df[["loan_status"]].copy()
#y = y.ravel()

#example: y = df_loans["bad"].ravel()

In [133]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [134]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [135]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [136]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [137]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [138]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [139]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [140]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)

simple_log_regression_bas = balanced_accuracy_score(y_test, y_pred)

balanced_accuracy_score(y_test, y_pred)

0.9854053094832482

In [141]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

simple_log_regression_confusion_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix(y_test, y_pred)

array([[  586,    14],
       [  110, 18674]])

In [142]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

simple_log_regression_classification_report = classification_report_imbalanced(y_test, y_pred)

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.98      0.99      0.90      0.99      0.97       600
   low_risk       1.00      0.99      0.98      1.00      0.99      0.97     18784

avg / total       0.99      0.99      0.98      0.99      0.99      0.97     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [143]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [144]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [145]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)

oversampling_bas = balanced_accuracy_score(y_test, y_pred)

balanced_accuracy_score(y_test, y_pred)

0.9951923622941511

In [146]:
# Display the confusion matrix
oversampling_confusion_matrix = confusion_matrix(y_test, y_pred)

confusion_matrix(y_test, y_pred)

array([[  598,     2],
       [  118, 18666]])

In [147]:
# Print the imbalanced classification report

oversampling_classification_report = classification_report_imbalanced(y_test, y_pred)

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       600
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



### SMOTE Oversampling

In [148]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)
from collections import Counter

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [149]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [150]:
# Calculated the balanced accuracy score

y_pred = model.predict(X_test_scaled)

SMOTE_oversampling_bas = balanced_accuracy_score(y_test, y_pred)

balanced_accuracy_score(y_test, y_pred)

0.9951923622941511

In [151]:
# Display the confusion matrix
SMOTE_oversampling_confusion_matrix = confusion_matrix(y_test, y_pred)

confusion_matrix(y_test, y_pred)

array([[  598,     2],
       [  118, 18666]])

In [152]:
# Print the imbalanced classification report
SMOTE_oversampling_classification_report = classification_report_imbalanced(y_test, y_pred)

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       600
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [153]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE
from imblearn.under_sampling import ClusterCentroids 

ros = ClusterCentroids(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)


# View the count of target classes with Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [154]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [155]:
# Calculate the balanced accuracy score

y_pred = model.predict(X_test_scaled)

undersampling_bas = balanced_accuracy_score(y_test, y_pred)

balanced_accuracy_score(y_test, y_pred)

0.992878691084611

In [156]:
# Display the confusion matrix
undersampling_confusion_matrix = confusion_matrix(y_test, y_pred)

confusion_matrix(y_test, y_pred)

array([[  595,     5],
       [  111, 18673]])

In [157]:
# Print the imbalanced classification report
undersampling_classification_report = classification_report_imbalanced(y_test, y_pred)

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       600
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18784

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [158]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train_scaled, y_train)
# View the count of target classes with Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [159]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [160]:
# Calculate the balanced accuracy score

y_pred = model.predict(X_test_scaled)

combination_sampling_bas = balanced_accuracy_score(y_test, y_pred)

balanced_accuracy_score(y_test, y_pred)

0.9951923622941511

In [163]:
# Display the confusion matrix
combination_sampling_confusion_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix(y_test, y_pred)

array([[  598,     2],
       [  118, 18666]])

In [164]:
# Print the imbalanced classification report
combination_sampling_classification_report = classification_report_imbalanced(y_test, y_pred)

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       600
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   Oversampling, SMOTE Oversampling, Combination Sampling, all had the same balanced accuracy score of 0.9951923622941511. 
       refer to '#1' below 

2. Which model had the best recall score?

    They all had the same total/ave recall score. However Simple Log Regression model had slighty worse out of them all due to high_risk and low_risk with 0.98 and 0.99.  
    
    This makes the Oversampling, SMOTE Oversampling, Combination Sampling models have all the highest, with at high_risk and low_risk recall scores at 1 and 0.99. 
        refer to '#2' below

3. Which model had the best geometric mean score?

    Oversampling, SMOTE Oversampling, Combination Sampling models have all the highest geo score (geometric mean score).
            refer to '#2' below


In [177]:
#1
print('Balanced accuracy score:')
print(f'simple_log_regression_bas {simple_log_regression_bas}')
print(f'oversampling_bas {oversampling_bas}')
print(f'SMOTE_oversampling_bas {SMOTE_oversampling_bas}')
print(f'undersampling_bas {undersampling_bas}')
print(f'combination_sampling_bas {combination_sampling_bas}')

Balanced accuracy score:
simple_log_regression_bas 0.9854053094832482
oversampling_bas 0.9951923622941511
SMOTE_oversampling_bas 0.9951923622941511
undersampling_bas 0.992878691084611
combination_sampling_bas 0.9951923622941511


In [186]:
#2
print('Classfication Report - Recall Score: \n')
print(f'simple_log_regression_classification_report \n {simple_log_regression_classification_report}')
print(f'oversampling_classification_report \n {oversampling_classification_report}')
print(f'SMOTE_oversampling_classification_report \n {SMOTE_oversampling_classification_report}')
print(f'undersampling_classification_report \n {undersampling_classification_report}')
print(f'combination_sampling_classification_report \n {combination_sampling_classification_report}')

Classfication Report - Recall Score: 

simple_log_regression_classification_report 
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.98      0.99      0.90      0.99      0.97       600
   low_risk       1.00      0.99      0.98      1.00      0.99      0.97     18784

avg / total       0.99      0.99      0.98      0.99      0.99      0.97     19384

oversampling_classification_report 
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       600
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384

SMOTE_oversampling_classification_report 
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       60